In [1]:
from fastT5 import  generate_onnx_representation


model_or_model_path = 't5-small'

# Step 1. convert huggingfaces t5 model to onnx
onnx_model_paths = generate_onnx_representation(model_or_model_path)

[2023-10-01 23:03:33,282] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


Exporting to onnx... |################################| 3/3


In [2]:
onnx_model_paths

(PosixPath('/home/venom/repo/fastT5/models/t5-small-encoder.onnx'),
 PosixPath('/home/venom/repo/fastT5/models/t5-small-decoder.onnx'),
 PosixPath('/home/venom/repo/fastT5/models/t5-small-init-decoder.onnx'))

In [3]:
from fastT5 import quantize

# Step 2. (recommended) quantize the converted model for fast inference and to reduce model size.
quant_model_paths = quantize(onnx_model_paths)

AttributeError: FLOAT8E4M3FN

In [ ]:
quant_model_paths

In [ ]:
import onnxruntime as ort

ort.get_all_providers()

In [ ]:
from fastT5 import get_onnx_runtime_sessions, OnnxT5

# step 3. setup onnx runtime
device = "CPU_FP32"
model_sessions = get_onnx_runtime_sessions(quant_model_paths, provider=["OpenVINOExecutionProvider"], provider_options=[{'device_type': 'CPU_FP32'}])

# step 4. get the onnx model
model = OnnxT5(model_or_model_path, model_sessions)

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("t5-small")

In [ ]:
text = """
Translate English to French: Lisa Marie Nowak (née Caputo, born May 10, 1963) is an American aeronautical engineer, convicted criminal, and former NASA astronaut and United States Navy officer. Nowak served as naval flight officer and test pilot in the Navy, and was selected by NASA for NASA Astronaut Group 16 in 1996, qualifying as a mission specialist in robotics. She flew in space aboard Space Shuttle Discovery during the STS-121 mission in July 2006, when she was responsible for operating the robotic arms of the shuttle and the International Space Station. In 2007, Nowak was involved in a highly publicized incident of criminal misconduct for which she eventually pled guilty to felony burglary and misdemeanor battery charges, resulting in her demotion from captain to commander, and termination by NASA and the Navy. Born in Washington, D.C., Nowak graduated from the United States Naval Academy in Annapolis, Maryland, in 1985. She was assigned to VAQ-34 at Naval Air Station Point Mugu, California, where she flew the EA-7L Corsair and ERA-3B Skywarrior. She earned a Master of Science degree in aeronautical engineering and a degree in aeronautical and astronautical engineering from the Naval Postgraduate School in Monterey, California. In 1993 she was selected to attend the U.S. Naval Test Pilot School at Naval Air Station Patuxent River, Maryland. After graduation, she remained at Patuxent River, flying in the F/A-18 Hornet and EA-6B Prowler. During her Navy career she logged over 1,500 hours in more than 30 aircraft and was awarded the Defense Meritorious Service Medal, the Navy Commendation Medal and the Navy Achievement Medal.In February 2007, Nowak was arrested in Orlando, Florida, after she accosted and pepper-sprayed Colleen Shipman, a U.S. Air Force captain romantically involved with astronaut William Oefelein, who had been in a relationship with Nowak. She was released on bail and initially pleaded not guilty to the charges, which included attempted kidnapping, burglary with assault, and battery. Subsequently, her assignment as an astronaut was terminated by NASA. In 2009, Nowak agreed to a plea deal with prosecutors and pleaded guilty to charges of felony burglary of a car and misdemeanor battery. She remained a Navy captain until the following year when a Naval Board of Inquiry voted unanimously to reduce her in rank to commander and to discharge her from the Navy under other than honorable conditions after 25 years of service. As of 2017, it was reported that she was working in the private sector in Texas. Early life and education Lisa Marie Caputo was born in Washington, D.C., on May 10, 1963, to Alfredo F. Caputo, a computer consultant, and Jane L.and termination by NASA and the Navy. Born in Washington, D.C., Nowak graduated from the United States Naval Academy in Annapolis, Maryland, in 1985. She was assigned to VAQ-34 at Naval Air Station Point Mugu, California, where she flew the EA-7L Corsair and ERA-3B Skywarrior. She earned a Master of Science degree in aeronautical engineering and a degree in aeronautical and astronautical engineering from the Naval Postgraduate School in Monterey, California. In 1993 she was selected to attend the U.S. Naval Test Pilot School at Naval Air Station Patuxent River, Maryland. After graduation, she remained at Patuxent River, flying in the F/A-18 Hornet and EA-6B Prowler. During her Navy career she logged over 1,500 hours in more than 30 aircraft and was awarded the Defense Meritorious Service Medal, the Navy Commendation Medal and the Navy Achievement Medal. In February 2007, Nowak was arrested in Orlando, Florida, after she accosted and pepper-sprayed Colleen Shipman, a U.S. Air Force captain romantically involved with astronaut William Oefelein, who had been in a relationship with Nowak. She was released on bail and initially pleaded not guilty to the charges, which included attempted kidnapping, burglary with assault, and battery. Subsequently, her assignment as an astronaut was terminated by NASA. In 2009, Nowak agreed to a plea deal with prosecutors and pleaded guilty to charges of felony burglary of a car and misdemeanor battery. She remained a Navy captain until the following year when a Naval Board of Inquiry voted unanimously to reduce her in rank to commander and to discharge her from the Navy under other than honorable conditions after 25 years of service. As of 2017, it was reported that she was working in the private sector in Texas. Early life and education Lisa Marie Caputo was born in Washington, D.C., on May 10, 1963, to Alfredo F. Caputo, a computer consultant, and Jane L"""

In [ ]:
token = tokenizer(text, return_tensors='pt', max_length=512, truncation=True)

In [ ]:
tokens = model.generate(input_ids=token['input_ids'],
               attention_mask=token['attention_mask'],)